In [1]:
%matplotlib inline

import xarray as xr
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from da_utils import (Forcings, perturb_forcings_ensemble, setup_output_dirs,
                      to_netcdf_forcing_file_compress, calculate_sm_noise_to_add_magnitude,
                      calculate_scale_n_whole_field, to_netcdf_state_file_compress,
                      calculate_max_soil_moist_domain, convert_max_moist_n_state)

In [2]:
# ===================================================== #
# Parameter setting
# ===================================================== #
lat = 34.6875
lon = -94.9375

# Root directory - all other paths will be under root_dir
root_dir = '/civil/hydro/ymao/data_assim/'

# --- Time --- #
start_time = pd.to_datetime('1980-01-01-00')
end_time = pd.to_datetime('1989-12-31-21')
start_year = start_time.year
end_year = end_time.year
state_times = pd.date_range(start_time, end_time, freq='D')

# --- Inputs --- #
# Orig. forcing netcdf basepath ('YYYY.nc' will be appended)
force_orig_nc = os.path.join(root_dir, 'forcing/vic/Newman/{}_{}/ens_100/force.'.format(lat, lon))
# Orig. history netcdf file
hist_orig_nc = os.path.join(
    root_dir,
    ('output/vic/ArkRed/openloop.1980_1989.Maurer_param/'
     'history/history.openloop.{}_{}.1980-01-01-00000.nc').format(lat, lon))
# Orig state basepath ('YYYYMMDD_SSSSS.nc' will be appended); initial state not included
state_orig_basepath = os.path.join(
    root_dir,
    'output/vic/ArkRed/openloop.1980_1989.Maurer_param/states/{}_{}/state.openloop.'.format(lat, lon))
# Initial state file
init_state_nc = os.path.join(
    root_dir,
    'output/vic/ArkRed/spinup.1949_1979.Maurer_param/states/state.{}_{}.19800101_00000.nc'.format(lat, lon))
# VIC global template file
vic_global_template_path = os.path.join(
    root_dir, 'control/vic/hyak.global.34.6875_-94.9375.template.Maurer_param.txt')

# --- Perturbation --- #
### prec. perturbation ###
# Number of prec. perturbation ensemble
N_prec = 3
# Standard deviation of prec. perturbation multiplier (fixed)
prec_std = 1
# Parameter in AR(1) process for prec. perturbation (fixed)
# prec_phi = 0
### State perturbation ###
# Number of state perturbation ensemble
N_state = 3
# Percentage of max value of each state to perturb (e.g., if
# state_perturb_sigma_percent = 5, then Gaussian noise with standard deviation
# = 5% of max soil moisture will be added as perturbation)
# state_perturb_sigma_percent is a list of percentage for each soil layer
# (e.g., if there are three soil layers, then an example of state_perturb_sigma_percent is: 20,1,0.5)
# NOTE: keep this consistent as in the dual correction system
state_perturb_sigma_percent = [5, 5, 0.5]

# --- Outputs --- #
output_dir = 'tools/error_correlation_synthetic/output/{}_{}'.format(lat, lon)

In [3]:
# ===================================================== #
# Load forcing and perturb precipitation (same multiplier for prec every day)
# ===================================================== #
# --- Load orig. forcing --- #
ds_force_orig = load_nc_file(force_orig_nc + '{}.nc', start_year, end_year)
# --- Set up output directory --- #
force_pert_basedir = setup_output_dirs(
    os.path.join(root_dir, output_dir),
    mkdirs=['perturbed_forcings'])['perturbed_forcings']
force_pert_noise_basedir = setup_output_dirs(
    force_pert_basedir,
    mkdirs=['corrcoef_0'])['corrcoef_0']

NameError: name 'load_nc_file' is not defined

In [ ]:
np.random.seed(1111)
pert_prec_cell_ensemble(N_prec, ds_force_orig, prec_std, force_pert_noise_basedir)

In [239]:
def pert_prec_cell_ensemble(N, ds_force_orig, prec_std, out_forcing_basedir):
    ''' Perturb precipitation to produce a ensemble of perturbed forcing.
        Only for one-grid-cell case.
    
    Parameters
    ----------
    N: <int>
        Ensemble size
    ds_force_orig: <xr.Dataset>
        Original VIC-format forcing
    prec_std: <float>
        Standard deviation of the precipitation perturbing multiplier
    '''
    for ens in range(N):
        print('Perturbing precipitation ens. {}...'.format(ens+1))
        pert_prec_cell(ens, ds_force_orig, prec_std, out_forcing_basedir)

In [243]:
def pert_prec_cell(ens, ds_force_orig, prec_std, out_forcing_basedir, seed=None):
    ''' Perturb precipitation to produce a perturbed forcing.
        Only for one-grid-cell case.

    Parameters
    ----------
    ens: <int>
        Index of ensemble (start from 0)
    ds_force_orig: <xr.Dataset>
        Original VIC-format forcing
    prec_std: <float>
        Standard deviation of the precipitation perturbing multiplier
    seed: <int or None>
        Seed for random number generator; this seed will only be used locally
        in this function and will not affect the upper-level code.
        None for not re-assign seed in this function, but using the global seed)
        Default: None
    out_forcing_basedir: <str>
        Base directory for output perturbed forcings;
        Subdirs "ens_<i>" will be created, where <i> is ensemble index, 1, ..., N
        File names will be: forc.YYYY.nc
    '''
    
    # --- Aggregate orig. prec to daily --- #
    da_prec_orig = ds_force_orig['PREC'].squeeze()  # [time]
    da_prec_orig_daily = da_prec_orig.groupby('time.date').sum()  # [day]
    
    # --- Generate multipliers for daily prec. --- #
    # Calculate mu and sigma for the lognormal distribution
    # (here mu and sigma are mean and std of the underlying normal dist.)
    mu = -0.5 * np.log(prec_std^2 + 1)
    sigma = np.sqrt(np.log(prec_std^2 + 1))
    if seed is None:
        multiplier_prec_daily = np.random.lognormal(
            mean=mu, sigma=sigma,
            size=(len(da_prec_orig_daily['date'])))
    else:
        rng = np.random.RandomState(seed)
        multiplier_prec_daily = rng.lognormal(
            mean=mu, sigma=sigma,
            size=(len(da_prec_orig_daily['date'])))
    # --- Distribute daily multipliers to sub-daily --- #
    da_multiplier_prec_daily = da_prec_orig_daily.copy(deep=True)
    da_multiplier_prec_daily[:] = multiplier_prec_daily
    da_multiplier_prec = da_prec_orig.copy(deep=True)
    times = [pd.to_datetime(time).date() for time in da_multiplier_prec['time'].values]
    for date, item in da_multiplier_prec.groupby('time.date'):
        time_ind = [time == date for time in times]
        da_multiplier_prec[time_ind] = da_multiplier_prec_daily.loc[date]
    # --- Perturb sub-daily prec --- #
    da_prec_pert = da_prec_orig * da_multiplier_prec
    # --- Save perturbed prec. to file --- #
    # Replace perturbed prec. in the original forcing data
    ds_force_pert = ds_force_orig.copy(deep=True)
    ds_force_pert['PREC'][:, 0, 0] = da_prec_pert
    # Set up ensemble subdir
    subdir_name = 'ens_{}'.format(ens+1)
    force_pert_ens_basedir = setup_output_dirs(
        out_forcing_basedir,
        mkdirs=[subdir_name])[subdir_name]
    for year, ds in ds_force_pert.groupby('time.year'):
        to_netcdf_forcing_file_compress(
            ds, os.path.join(force_pert_ens_basedir, 'force.{}.nc'.format(year)))

In [14]:
def pert_state_cell_ensemble(N, states_orig, scale_n_nloop, out_state_basedir, da_max_moist_n):
    ''' Perturb precipitation to produce a ensemble of perturbed forcing.
        Only for one-grid-cell case.
    
    Parameters
    ----------
    N: <int>
        Ensemble size
    states_orig: <list>
        List of original VIC-format states
    scale_n_nloop: <np.array>
        Standard deviation of noise to add for the whole field.
        Dimension: [nloop, n] (where nloop = lat * lon = 1)
    out_state_basedir: <str>
        Base directory for output perturbed states;
        Subdirs "ens_<i>" will be created, where <i> is ensemble index, 1, ..., N
        File names will be: state.YYYY_SSSSS.nc
    da_max_moist_n: <xarray.DataArray>
            Maximum soil moisture for the whole domain and each tile
            [unit: mm]. Soil moistures above maximum after perturbation will
            be reset to maximum value.
            Dimension: [lat, lon, n]
    '''
    
    for ens in range(N):
        print('Perturbing states ens. {}...'.format(ens+1))
        pert_state_cell(ens, states_orig, scale_n_nloop, out_state_basedir, da_max_moist_n)

In [17]:
def pert_state_cell(ens, states_orig, scale_n_nloop, out_state_basedir, da_max_moist_n, seed=None):
    ''' Perturb precipitation to produce a perturbed forcing.
        Only for one-grid-cell case.
        
    Parameters
    ----------
    ens: <int>
        Index of ensemble (start from 0)
    states_orig: <list>
        List of original VIC-format states
    scale_n_nloop: <np.array>
        Standard deviation of noise to add for the whole field.
        Dimension: [nloop, n] (where nloop = lat * lon = 1)
    out_state_basedir: <str>
        Base directory for output perturbed states;
        Subdirs "ens_<i>" will be created, where <i> is ensemble index, 1, ..., N
        File names will be: state.YYYY_SSSSS.nc
    seed: <int or None>
        Seed for random number generator; this seed will only be used locally
        in this function and will not affect the upper-level code.
        None for not re-assign seed in this function, but using the global seed)
        Default: None
    da_max_moist_n: <xarray.DataArray>
            Maximum soil moisture for the whole domain and each tile
            [unit: mm]. Soil moistures above maximum after perturbation will
            be reset to maximum value.
            Dimension: [lat, lon, n]
    '''
    
    # Generate one N(0, 1) random noise for each time step
    if seed is None:
        noise_standard = np.random.normal(0, 1, [len(state_times), 1])  # [time, 1]
    else:
        rng = np.random.RandomState(seed)
        noise_standard = rng.normal(0, 1, [len(state_times), 1])  # [time, 1]
    # Scale noise for each layer (and uniform for each tile)
    noise_scaled = np.dot(noise_standard, scale_n_nloop)  # [time, n]
    noise_scaled = noise_scaled.reshape(
        [len(state_times), len(states_orig[0]['nlayer']), len(states_orig[0]['veg_class']),
         len(states_orig[0]['snow_band'])])  # [time, nlayer, nveg, nsnow]
    noise_scaled = np.rollaxis(noise_scaled, 1, 4)  # [time, nveg, nsnow, nlayer]
    # Set up ensemble subdir
    subdir_name = 'ens_{}'.format(ens+1)
    state_pert_ens_basedir = setup_output_dirs(
        state_pert_noise_basedir,
        mkdirs=[subdir_name])[subdir_name]
    # Add noise to the orig. states and save to file
    for i, time in enumerate(state_times):
        # Replace perturbed states in the orig. state
        ds_states_pert = states_orig[i].copy(deep=True)
        ds_states_pert['STATE_SOIL_MOISTURE'][:, :, :, 0, 0] += noise_scaled[i, :, :, :]
        # Adjust negative SM to zero
        sm_new = ds_states_pert['STATE_SOIL_MOISTURE'].values
        sm_new[sm_new<0] = 0
        # Reset perturbed soil moistures above maximum to maximum
        max_moist = da_max_moist_n.values
        print(max_moist.shape)
        print(sm_new.shape)
        sm_new[(sm_new>max_moist)] = max_moist[(sm_new>max_moist)]
        ds_states_pert['STATE_SOIL_MOISTURE'] = sm_new
        # Save
        to_netcdf_state_file_compress(
            ds_states_pert,
            os.path.join(state_pert_ens_basedir,
                         'state.{}_{:05d}.nc'.format(time.strftime('%Y%m%d'), time.second)))

In [10]:
# ===================================================== #
# Load state files and perturb SM
# ===================================================== #
# Load states
states_orig = []
# Load initial state
states_orig.append(xr.open_dataset(init_state_nc))
# Load all times of states
for time in state_times[1:]:
    state_nc = state_orig_basepath + time.strftime('%Y%m%d') + '_' + '{:05d}'.format(time.second) + '.nc'
    states_orig.append(xr.open_dataset(state_nc))

In [11]:
# --- Set up output directory --- #
state_pert_basedir = setup_output_dirs(
    os.path.join(root_dir, output_dir),
    mkdirs=['perturbed_states'])['perturbed_states']
state_pert_noise_basedir = setup_output_dirs(
    state_pert_basedir,
    mkdirs=['corrcoef_0'])['corrcoef_0']

In [12]:
# --- Perturb all-layer SM states --- #
# --- (Same perturbation for all tiles/layers, with different magtinude for each layer) --- #
# Calculate perturbation scale for each layer
da_scale = calculate_sm_noise_to_add_magnitude(
    vic_history_path=hist_orig_nc,
    sigma_percent=state_perturb_sigma_percent)
# Extract maximum moisture for each layer
nveg = len(states_orig[0]['veg_class'])
nsnow = len(states_orig[0]['snow_band'])
da_max_moist = calculate_max_soil_moist_domain(vic_global_template_path)
da_max_moist_n = convert_max_moist_n_state(da_max_moist, nveg, nsnow)
scale_n_nloop = calculate_scale_n_whole_field(
    da_scale, nveg, nsnow)  # [lat*lon=1, n=nlayer*nveg*nsnow]

In [18]:
# --- Perturb states for all ensemble members --- #
pert_state_cell_ensemble(N_state, states_orig, scale_n_nloop, state_pert_noise_basedir, da_max_moist_n)

Perturbing states ens. 1...
(1, 1, 36)
(12, 1, 3, 1, 1)


/usr/lusers/yixinmao/.conda/envs/vic5/lib/python3.5/site-packages/ipykernel/__main__.py:54: RuntimeWarning: invalid value encountered in less
/usr/lusers/yixinmao/.conda/envs/vic5/lib/python3.5/site-packages/ipykernel/__main__.py:59: RuntimeWarning: invalid value encountered in greater


IndexError: too many indices for array

In [5]:
def load_nc_file(nc_file, start_year, end_year):
    ''' Loads in nc files for all years.

    Parameters
    ----------
    nc_file: <str>
        netCDF file to load, with {} to be substituted by YYYY
    start_year: <int>
        Start year
    end_year: <int>
        End year

    Returns
    ----------
    ds_all_years: <xr.Dataset>
        Dataset of all years
    '''

    list_ds = []
    for year in range(start_year, end_year+1):
        # Load data
        fname = nc_file.format(year)
        ds = xr.open_dataset(fname)
        list_ds.append(ds)
        # Concat all years
        ds_all_years = xr.concat(list_ds, dim='time')

    return ds_all_years